# Benchmarking on the open datasets Madelon, Gina Priori and Scene, and UK Biobank Cancer

## Overview
This notebook does feature selection by using seven feature selection methods tested (LLpowershap, Powershap, Borutashap, Shapicant, top 3%, Chi² squared and F test).

Adapted from the notebook by the authors of the Powershap python package available at https://github.com/predict-idlab/powershap

Orignal Notebook: https://github.com/predict-idlab/powershap/blob/main/examples/Benchmark_datasets.ipynb



## Dataset Information
- Madelon, https://www.openml.org/d/1485
- Gina priori, https://www.openml.org/d/1042
- Scene, https://www.openml.org/d/312
- The UK Biobank is an open access resource and bona fide researchers can apply to use the UK Biobank dataset by registering and applying at https://www.ukbiobank.ac.uk/enable-yourresearch/register.


## Notes.


# Imports and functions

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
from numpy.random import RandomState
import pandas as pd
from math import sqrt
from sklearn.model_selection import train_test_split, KFold, StratifiedKFold, LeaveOneOut
from sklearn import feature_selection as fs
from sklearn import metrics as me
from sklearn.metrics import classification_report,auc,r2_score,matthews_corrcoef,roc_auc_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import chi2,f_classif,f_regression
from catboost import CatBoostRegressor,CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from catboost.utils import get_roc_curve
from sklearn.metrics import precision_recall_curve
from catboost import Pool, cv
import shap
from scipy import stats
from scipy.optimize import curve_fit
import copy
from tabulate import tabulate
from tqdm import tqdm
from BorutaShap import BorutaShap
from powershap  import PowerShap
import shapicant

import time
import shap

import ast

In [3]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score,classification_report,auc,r2_score,matthews_corrcoef

## This cross_validation is different from the standard cross validation, because the a priori dataset is only tested on actual a priori samples
# Inp_db:pandas_DataFrame = input database (contains labels and features), should contain heroi2c numbers as IDs, previous concentrations
# Features:List = features to use
# Folds:int = amount of folds
# RS:int = Random state
# Output:dict = dictionary with the results of each fold
def benchmark_classification_cross_validation(Model,Inp_db,index_col,folds,RS,features,target_col,disable_tqdm_output=False):
    kf = KFold(n_splits=folds,shuffle=True,random_state=RS)

    scores_cv_train = {"AUC":np.array([]),
                 "MCC":np.array([]),      
                 "ACCURACY":np.array([]),
                 "RECALL":np.array([]),
                 "F1":np.array([]),
                 "PRECISION":np.array([]),
                 "AUC_PRC":np.array([])}
    
    scores_cv_test = {"AUC":np.array([]),
                 "MCC":np.array([]),      
                 "ACCURACY":np.array([]),
                 "RECALL":np.array([]),
                 "F1":np.array([]),
                 "PRECISION":np.array([]),
                 "AUC_PRC":np.array([])}
    
    for CV_train_idx, CV_test_idx in tqdm(kf.split(Inp_db[index_col].unique()),disable=disable_tqdm_output):  
        ## Split per patient (to avoid data leakage)
        X_CV_train = Inp_db[Inp_db[index_col].isin(Inp_db[index_col].unique()[CV_train_idx])]
        X_CV_test = Inp_db[Inp_db[index_col].isin(Inp_db[index_col].unique()[CV_test_idx])]
        
        Y_CV_train = X_CV_train[target_col].values
        Y_CV_test = X_CV_test[target_col].values
        
        ## Extract the required features
        X_CV_train_feat = X_CV_train[features]
        X_CV_test_feat = X_CV_test[features]
        
        try:
            Model.fit(X_CV_train_feat,Y_CV_train,eval_set=(X_CV_test[features],Y_CV_test))
            #print('successfully executed with eval_set')
        except:
            Model.fit(X_CV_train_feat,Y_CV_train,eval_set=[(X_CV_test[features],Y_CV_test)], verbose=False)
            # Model.fit(X_CV_train_feat,Y_CV_train)
        
        Y_CV_predict_test = Model.predict(X_CV_test_feat)
        Y_CV_predict_train = Model.predict(X_CV_train_feat)
        y_cv_train_pred = Model.predict_proba(X_CV_train_feat)[:,1]
        y_cv_test_pred = Model.predict_proba(X_CV_test_feat)[:,1]

        def get_auc_prc(ys, y_probs):
            precision, recall, _ = precision_recall_curve(ys, y_probs)
            return auc(recall, precision)
       
        
        train_results = classification_report(Y_CV_train,Model.predict(X_CV_train_feat),output_dict=True)
        test_results = classification_report(Y_CV_test,Model.predict(X_CV_test_feat),output_dict=True)
        
        scores_cv_train["AUC"]=np.append(scores_cv_train["AUC"],roc_auc_score(Y_CV_train,y_cv_train_pred))
        scores_cv_test["AUC"]=np.append(scores_cv_test["AUC"],roc_auc_score(Y_CV_test,y_cv_test_pred))

        scores_cv_train["AUC_PRC"]=np.append(scores_cv_train["AUC_PRC"],get_auc_prc(Y_CV_train,y_cv_train_pred))
        scores_cv_test["AUC_PRC"]=np.append(scores_cv_test["AUC_PRC"],get_auc_prc(Y_CV_test,y_cv_test_pred))        
    
        scores_cv_train["MCC"]=np.append(scores_cv_train["MCC"],matthews_corrcoef(Y_CV_train,Model.predict(X_CV_train_feat)))
        scores_cv_test["MCC"]=np.append(scores_cv_test["MCC"],matthews_corrcoef(Y_CV_test,Model.predict(X_CV_test_feat)))
        
        scores_cv_train["ACCURACY"]=np.append(scores_cv_train["ACCURACY"],train_results["accuracy"])
        scores_cv_test["ACCURACY"]=np.append(scores_cv_test["ACCURACY"],test_results["accuracy"])

        scores_cv_train["RECALL"]=np.append(scores_cv_train["RECALL"],train_results['weighted avg']['recall'])
        scores_cv_test["RECALL"]=np.append(scores_cv_test["RECALL"],test_results['weighted avg']['recall'])

        scores_cv_train["F1"]=np.append(scores_cv_train["F1"],train_results['weighted avg']['f1-score'])
        scores_cv_test["F1"]=np.append(scores_cv_test["F1"],test_results['weighted avg']['f1-score'])

        scores_cv_train["PRECISION"]=np.append(scores_cv_train["PRECISION"],train_results['weighted avg']['precision'])
        scores_cv_test["PRECISION"]=np.append(scores_cv_test["PRECISION"],test_results['weighted avg']['precision'])

    #for key in scores_cv_train:
    #    scores_cv_train[key]=[np.mean(scores_cv_train[key][0]),np.std(scores_cv_train[key][0])]
    #    scores_cv_test[key]=[np.mean(scores_cv_test[key][0]),np.std(scores_cv_test[key][0])]

    return scores_cv_train,scores_cv_test


def benchmark_classification_loocv(Model, Inp_db, index_col, RS, features, target_col, disable_tqdm_output=False):
    loo = LeaveOneOut()

    all_y_true_train, all_y_pred_train, all_y_true_test, all_y_pred_test = [], [], [], []

    for train_idx, test_idx in tqdm(loo.split(Inp_db[index_col].unique()), disable=disable_tqdm_output):
        ## Split per patient (to avoid data leakage)
        X_CV_train = Inp_db[Inp_db[index_col].isin(Inp_db[index_col].unique()[train_idx])]
        X_CV_test = Inp_db[Inp_db[index_col].isin(Inp_db[index_col].unique()[test_idx])]

        Y_CV_train = X_CV_train[target_col].values
        Y_CV_test = X_CV_test[target_col].values

        ## Extract the required features
        X_CV_train_feat = X_CV_train[features]
        X_CV_test_feat = X_CV_test[features]

        try:
            Model.fit(X_CV_train_feat, Y_CV_train, eval_set=(X_CV_test[features], Y_CV_test))
        except:
            Model.fit(X_CV_train_feat, Y_CV_train)

        Y_CV_predict_train = Model.predict(X_CV_train_feat)
        y_cv_train_pred = Model.predict_proba(X_CV_train_feat)[:, 1]

        Y_CV_predict_test = Model.predict(X_CV_test_feat)
        y_cv_test_pred = Model.predict_proba(X_CV_test_feat)[:, 1]

        all_y_true_train.extend(Y_CV_train)
        all_y_pred_train.extend(y_cv_train_pred)

        all_y_true_test.extend(Y_CV_test)
        all_y_pred_test.extend(y_cv_test_pred)

    scores_cv_train = {
        "AUC": roc_auc_score(all_y_true_train, all_y_pred_train),
        "MCC": matthews_corrcoef(all_y_true_train, np.round(all_y_pred_train)),
        "ACCURACY": np.mean(all_y_true_train == np.round(all_y_pred_train)),
        "RECALL": classification_report(all_y_true_train, np.round(all_y_pred_train), output_dict=True)['weighted avg']['recall'],
        "F1": classification_report(all_y_true_train, np.round(all_y_pred_train), output_dict=True)['weighted avg']['f1-score'],
        "PRECISION": classification_report(all_y_true_train, np.round(all_y_pred_train), output_dict=True)['weighted avg']['precision']
    }

    scores_cv_test = {
        "AUC": roc_auc_score(all_y_true_test, all_y_pred_test),
        "MCC": matthews_corrcoef(all_y_true_test, np.round(all_y_pred_test)),
        "ACCURACY": np.mean(all_y_true_test == np.round(all_y_pred_test)),
        "RECALL": classification_report(all_y_true_test, np.round(all_y_pred_test), output_dict=True)['weighted avg']['recall'],
        "F1": classification_report(all_y_true_test, np.round(all_y_pred_test), output_dict=True)['weighted avg']['f1-score'],
        "PRECISION": classification_report(all_y_true_test, np.round(all_y_pred_test), output_dict=True)['weighted avg']['precision']
    }

    return scores_cv_train, scores_cv_test


In [4]:
from sklearn.model_selection import KFold,StratifiedKFold
from sklearn.metrics import roc_auc_score,classification_report,auc,r2_score,matthews_corrcoef

## This cross_validation is different from the standard cross validation, because the a priori dataset is only tested on actual a priori samples
# Inp_db:pandas_DataFrame = input database (contains labels and features), should contain heroi2c numbers as IDs, previous concentrations
# Features:List = features to use
# Folds:int = amount of folds
# RS:int = Random state
# Output:dict = dictionary with the results of each fold
def test_bootstrap_eval_class(Model,test_df,bootstrap_its,RS,features,target_col,disable_tqdm_output=False):
    scores_cv_test = {"AUC":np.array([]),
                 "MCC":np.array([]),      
                 "ACCURACY":np.array([]),
                 "RECALL":np.array([]),
                 "F1":np.array([]),
                 "PRECISION":np.array([]),
                 "AUC_PRC": np.array([])}
    
    for i in tqdm(range(bootstrap_its)):
        sampled_df = test_df.sample(n=int(0.66*len(test_df)),replace=True,random_state=i)

        Y_CV_test = sampled_df[target_col].values
        ## Extract the required features
        X_CV_test_feat = sampled_df[features].values

        Y_CV_predict_test = Model.predict(X_CV_test_feat)
        y_cv_test_pred = Model.predict_proba(X_CV_test_feat)[:,1]

        test_results = classification_report(Y_CV_test,Y_CV_predict_test,output_dict=True)
        
        def get_auc_prc(ys, y_probs):
            precision, recall, _ = precision_recall_curve(ys, y_probs)
            auc_prc = auc(recall, precision)
            return auc_prc        

        scores_cv_test["AUC"]=np.append(scores_cv_test["AUC"],roc_auc_score(Y_CV_test,y_cv_test_pred))
        scores_cv_test["AUC_PRC"]=np.append(scores_cv_test["AUC_PRC"],get_auc_prc(Y_CV_test,y_cv_test_pred))
        scores_cv_test["MCC"]=np.append(scores_cv_test["MCC"],matthews_corrcoef(Y_CV_test,Y_CV_predict_test))
        scores_cv_test["ACCURACY"]=np.append(scores_cv_test["ACCURACY"],test_results["accuracy"])
        scores_cv_test["RECALL"]=np.append(scores_cv_test["RECALL"],test_results['weighted avg']['recall'])
        scores_cv_test["F1"]=np.append(scores_cv_test["F1"],test_results['weighted avg']['f1-score'])
        scores_cv_test["PRECISION"]=np.append(scores_cv_test["PRECISION"],test_results['weighted avg']['precision'])

    return scores_cv_test

# MADELON dataset

## Read dataset

In [83]:
current_db = pd.read_csv("../data/madelon.csv", index_col=0)
current_db = current_db.reset_index()
train_idx,val_idx = train_test_split(current_db["index"],test_size=0.25,random_state = 1)
current_db_train = current_db[current_db["index"].isin(train_idx)]
current_db_test = current_db[current_db["index"].isin(val_idx)]

target_col = "Class"
Index_col = "index"

# Benchmarking

In [84]:
durations = {}
sub = 'xgb_250_git'
ds = 'madelon'

## Top 3 percent

In [13]:
n_iterations = 100

shap_vals_test = []
start_time = time.time()
for i in tqdm(range(n_iterations)):

    Inp_db = current_db_train.copy(deep=True)
    train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = i)
    
    X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]
    Y_val = Inp_db[Inp_db[Index_col].isin(val_idx)][target_col]

    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0)
    model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], verbose=False)
    explainer = shap.Explainer(model)
    shap_vals = explainer(X_val, Y_val).values
    shap_vals_mean = np.abs(shap_vals).mean(axis=0)
    shap_vals_test.append(shap_vals_mean)
    
durations['top_3_percent'] = time.time() - start_time
shap_val_summary = pd.DataFrame(data=np.array(shap_vals_test).mean(axis=0), columns=['shap'], index=current_db_train.columns[1:-1]).sort_values(by='shap',ascending=False)
selected_features = shap_val_summary.iloc[:int(len(current_db_train.columns[1:-1])*0.03),:].index.tolist()

with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'wt') as file:
    file.write(str(selected_features))
               
print(selected_features)
print(len(selected_features))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:57<00:00,  1.74it/s]

['V476', 'V339', 'V379', 'V154', 'V49', 'V106', 'V29', 'V319', 'V443', 'V242', 'V454', 'V282', 'V286', 'V473', 'V5']
15


## Powershap

In [14]:
start_time = time.time()
selector = PowerShap(
    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='powershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['powershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_PowerSHAP_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting powershap
Automatic mode enabled: Finding the minimal required powershap iterations for significance of 0.01.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: powershap requires 11 iterations; Adding 1  powershap iterations.


  0%|          | 0/1 [00:00<?, ?it/s]

Done!
selected features: ['V106', 'V154', 'V242', 'V29', 'V319', 'V339', 'V379', 'V443', 'V454', 'V49']
10


## LLpowershap

In [47]:
start_time = time.time()
selector = PowerShap(model=XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='llpowershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['LLpowershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_LLpowershap_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting llpowershap
Automatic mode enabled: Finding the minimal required llpowershap iterations for significance of 0.01.


  0%|          | 0/20 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 37  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 50  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 57  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Done!
selected features: ['V106', 'V129', 'V154', 'V242', 'V282', 'V286', 'V29', 'V319', 'V337', 'V339', 'V379', 'V434', 'V443', 'V452', 'V454', 'V473', 'V476', 'V49', 'V494', 'V5', 'V65']
21


## Borutashap

In [16]:
model = XGBClassifier(n_estimators=250, verbosity=0)#, #early_stopping_rounds=25)

# if classification is False it is a Regression problem
start_time = time.time()
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=current_db_train[list(current_db_train.columns.values[1:-1])], y=current_db_train[target_col], sample=False,
                        train_or_test = 'test', normalize=True,verbose=True)
dur = time.time() - start_time
durations['borutashap'] = dur

subset = Feature_Selector.Subset()
subset.to_csv(f'../results/{sub}/{ds}_borutashap_results.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

6 attributes confirmed important: ['V339', 'V242', 'V476', 'V286', 'V379', 'V29']
490 attributes confirmed unimportant: ['V202', 'V133', 'V119', 'V193', 'V244', 'V460', 'V252', 'V449', 'V440', 'V287', 'V358', 'V45', 'V466', 'V465', 'V328', 'V16', 'V280', 'V33', 'V459', 'V289', 'V402', 'V74', 'V229', 'V427', 'V483', 'V352', 'V430', 'V116', 'V384', 'V438', 'V26', 'V490', 'V348', 'V419', 'V187', 'V189', 'V15', 'V63', 'V46', 'V141', 'V150', 'V304', 'V39', 'V134', 'V346', 'V190', 'V266', 'V370', 'V269', 'V114', 'V146', 'V93', 'V101', 'V52', 'V130', 'V20', 'V226', 'V89', 'V2', 'V159', 'V301', 'V94', 'V123', 'V192', 'V407', 'V210', 'V240', 'V292', 'V171', 'V41', 'V224', 'V306', 'V399', 'V351', 'V390', 'V182', 'V334', 'V96', 'V369', 'V186', 'V296', 'V491', 'V340', 'V175', 'V247', 'V347', 'V257', 'V18', 'V128', 'V378', 'V350', 'V54', 'V106', 'V27', 'V72', 'V111', 'V471', 'V302', 'V403', 'V487', 'V333', 'V113', 'V86', 'V243', 'V416', 'V422', 'V151', 'V47', 'V120', 'V428', 'V383', 'V19', 'V446', 

## Shapicant

In [17]:
Inp_db = current_db_train.copy(deep=True)
train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = 0)

X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]

model = XGBClassifier(n_estimators=250,  verbosity=0) #,early_stopping_rounds=25)

start_time = time.time()
# This is the class (not its instance) of SHAP's TreeExplainer
explainer_type = shap.TreeExplainer

# Use PandasSelector with 100 iterations
selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

# Run the feature selection
# If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
# We can also provide additional parameters to the underlying estimator's fit method through estimator_params
selector.fit(X_train, Y_train, X_validation=X_val)#, estimator_params={"categorical_feature": None})

dur = time.time() - start_time
durations['shapicant'] = dur

# Just get the features list
selected_features = selector.get_features()

# We can also get the p-values as pandas Series
p_values = selector.p_values_

np.array(selected_features)
with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'wt') as file:
    file.write(str(selected_features))

Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [02:31<00:00,  1.52s/it]


In [19]:
with open(f'../results/{sub}/{ds}_shap_methods_durations.txt', 'wt') as file:
    file.write(str(durations))

# 10 fold cross validation

In [85]:
import warnings
warnings.filterwarnings('ignore')

method_names = {'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features', 'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    for folds in [10]:
    
        results_df : pd.DataFrame = None
        
        for method in method_names:
            if method == 'top_3_percent':
                with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                    s = file.read()
                selected_feautres = ast.literal_eval(s)
            elif method == 'llpowershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
            elif method == 'borutashap':
                df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
                selected_features = df.columns
            elif method == 'chi_squared':
                selected_features = list(current_db_train.columns.values[1:-1][np.where(chi2(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
            elif method == 'f_test':
                selected_features = list(current_db_train.columns.values[1:-1][np.where(f_classif(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
            elif method == 'powershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
            elif method == 'shapicant':
                with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                    s = file.read()
                selected_features = ast.literal_eval(s)
            elif method == 'default':
                selected_features = list(current_db.columns.values[1:-1])
        
            model = copy.deepcopy(models[m])
            scores_cv_train,scores_cv_test = benchmark_classification_cross_validation(Model = model,Inp_db = current_db_train.copy(deep=True),index_col=Index_col,folds=folds,RS=1,features = selected_features,target_col = target_col)
            
            print(method_names[method])
            print("TRAIN")
            for key in scores_cv_train:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_train[key]),4))+" ("+str(np.round(np.std(scores_cv_train[key]),4))+")")
            print(50*"-")
            print("TEST")
            for key in scores_cv_test:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),4))+" ("+str(np.round(np.std(scores_cv_test[key]),4))+")")
            print(50*"=")
        
            df_train = pd.DataFrame(scores_cv_train)
            df_train['data'] = 'train'
            df_train['method'] = method
        
            df_test = pd.DataFrame(scores_cv_test)
            df_test['data'] = 'test'
            df_test['method'] = method
        
            results_df = pd.concat([results_df, df_train, df_test])
        
        results_df.to_csv(f'../results/{sub}/{ds}_{folds}_fold_cross_val_results_with_{m}.csv', index=False)

        

10it [00:05,  1.72it/s]


LLpowershap
TRAIN
AUC: 0.9977 (0.002)
MCC: 0.9552 (0.0194)
ACCURACY: 0.9776 (0.0097)
RECALL: 0.9776 (0.0097)
F1: 0.9776 (0.0097)
PRECISION: 0.9776 (0.0097)
AUC_PRC: 0.9978 (0.0019)
--------------------------------------------------
TEST
AUC: 0.947 (0.0153)
MCC: 0.7691 (0.0321)
ACCURACY: 0.8841 (0.0159)
RECALL: 0.8841 (0.0159)
F1: 0.8841 (0.016)
PRECISION: 0.8854 (0.0163)
AUC_PRC: 0.9421 (0.0227)


10it [00:04,  2.10it/s]


powershap
TRAIN
AUC: 0.9971 (0.0017)
MCC: 0.9487 (0.0194)
ACCURACY: 0.9744 (0.0097)
RECALL: 0.9744 (0.0097)
F1: 0.9744 (0.0097)
PRECISION: 0.9744 (0.0097)
AUC_PRC: 0.9973 (0.0016)
--------------------------------------------------
TEST
AUC: 0.9436 (0.0144)
MCC: 0.7568 (0.0394)
ACCURACY: 0.8779 (0.0193)
RECALL: 0.8779 (0.0193)
F1: 0.8779 (0.0193)
PRECISION: 0.8794 (0.0198)
AUC_PRC: 0.9311 (0.027)


10it [00:03,  2.84it/s]


...

Chi²
TRAIN
AUC: 0.9987 (0.0023)
MCC: 0.9795 (0.0289)
ACCURACY: 0.9897 (0.0145)
RECALL: 0.9897 (0.0145)
F1: 0.9897 (0.0145)
PRECISION: 0.9898 (0.0145)
AUC_PRC: 0.9988 (0.0022)
--------------------------------------------------
TEST
AUC: 0.8904 (0.0225)
MCC: 0.6139 (0.0573)
ACCURACY: 0.8067 (0.0288)
RECALL: 0.8067 (0.0288)
F1: 0.8066 (0.0289)
PRECISION: 0.8078 (0.029)
AUC_PRC: 0.8872 (0.0355)


10it [00:01,  7.59it/s]


F Test
TRAIN
AUC: 0.9994 (0.0013)
MCC: 0.9871 (0.0203)
ACCURACY: 0.9936 (0.0102)
RECALL: 0.9936 (0.0102)
F1: 0.9936 (0.0102)
PRECISION: 0.9936 (0.0102)
AUC_PRC: 0.9995 (0.0012)
--------------------------------------------------
TEST
AUC: 0.9031 (0.0246)
MCC: 0.66 (0.0599)
ACCURACY: 0.8297 (0.0296)
RECALL: 0.8297 (0.0296)
F1: 0.8297 (0.0297)
PRECISION: 0.8308 (0.0304)
AUC_PRC: 0.9013 (0.0274)


10it [00:01,  7.59it/s]

top_3_percent
TRAIN
AUC: 0.9994 (0.0013)
MCC: 0.9871 (0.0203)
ACCURACY: 0.9936 (0.0102)
RECALL: 0.9936 (0.0102)
F1: 0.9936 (0.0102)
PRECISION: 0.9936 (0.0102)
AUC_PRC: 0.9995 (0.0012)
--------------------------------------------------
TEST
AUC: 0.9031 (0.0246)
MCC: 0.66 (0.0599)
ACCURACY: 0.8297 (0.0296)
RECALL: 0.8297 (0.0296)
F1: 0.8297 (0.0297)
PRECISION: 0.8308 (0.0304)
AUC_PRC: 0.9013 (0.0274)


# Bootstrap testing

In [63]:

method_names = {'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features', 'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    
    results_df : pd.DataFrame = None
    
    for method in method_names:
        if method == 'top_3_percent':
            with open(f'../results/{sub}/madelon_top_3_percent_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'llpowershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
        elif method == 'borutashap':
            df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
            selected_features = df.columns
        elif method == 'chi_squared':
            selected_features = list(current_db_train.columns.values[1:-1][np.where(chi2(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
        elif method == 'f_test':
            selected_features = list(current_db_train.columns.values[1:-1][np.where(f_classif(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
        elif method == 'powershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
        elif method == 'shapicant':
            with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'default':
            selected_features = list(current_db.columns.values[1:-1])

    
        X_train = current_db_train[selected_features]
        Y_train = current_db_train[target_col]
        
        X_test = current_db_test[selected_features]
        Y_test = current_db_test[target_col]

        model = copy.deepcopy(models[m])
        model.fit(X_train, Y_train)

        scores_cv_test = test_bootstrap_eval_class(Model = model, test_df = current_db_test.copy(deep=True),bootstrap_its=1000,RS=1,features = selected_features,target_col = target_col)

        print(method)
        print("TEST")
        for key in scores_cv_test:
            print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),3))+" ("+str(np.round(np.std(scores_cv_test[key]),3))+")")
    
        df_test = pd.DataFrame(scores_cv_test)
        df_test['data'] = 'test'
        df_test['method'] = method
    
        results_df = pd.concat([results_df, df_test])
    
    results_df.to_csv(f'../results/{sub}/{ds}_1000_bootstrap_results_with_{m}.csv', index=False)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 96.12it/s]


llpowershap
TEST
AUC: 0.955 (0.009)
MCC: 0.77 (0.031)
ACCURACY: 0.885 (0.015)
RECALL: 0.885 (0.015)
F1: 0.885 (0.015)
PRECISION: 0.886 (0.015)
AUC_PRC: 0.95 (0.012)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 99.67it/s]


powershap
TEST
AUC: 0.953 (0.009)
MCC: 0.766 (0.031)
ACCURACY: 0.882 (0.015)
RECALL: 0.882 (0.015)
F1: 0.882 (0.015)
PRECISION: 0.885 (0.015)
AUC_PRC: 0.948 (0.012)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 100.56it/s]


...

chi_squared
TEST
AUC: 0.906 (0.014)
MCC: 0.66 (0.036)
ACCURACY: 0.829 (0.018)
RECALL: 0.829 (0.018)
F1: 0.829 (0.018)
PRECISION: 0.832 (0.018)
AUC_PRC: 0.899 (0.019)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 70.78it/s]


f_test
TEST
AUC: 0.926 (0.012)
MCC: 0.669 (0.034)
ACCURACY: 0.832 (0.018)
RECALL: 0.832 (0.018)
F1: 0.832 (0.018)
PRECISION: 0.838 (0.017)
AUC_PRC: 0.923 (0.014)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 70.40it/s]


top_3_percent
TEST
AUC: 0.943 (0.011)
MCC: 0.726 (0.032)
ACCURACY: 0.863 (0.016)
RECALL: 0.863 (0.016)
F1: 0.863 (0.016)
PRECISION: 0.864 (0.016)
AUC_PRC: 0.936 (0.015)


# GINA PRIORI dataset

## Read dataset

In [86]:
current_db = pd.read_csv("../data/gina_priori.csv", index_col=0)
current_db = current_db.reset_index()
#current_db = current_db.iloc[:100, :]
current_db.loc[current_db.label==-1,"label"]=0
train_idx,val_idx = train_test_split(current_db["index"],test_size=0.25,random_state = 1)
current_db_train = current_db[current_db["index"].isin(train_idx)]
current_db_test = current_db[current_db["index"].isin(val_idx)]

target_col = "label"
Index_col = "index"

In [87]:
durations = {}
sub = 'xgb_250_git'
ds = 'gina_priori'

## Top 3 percent

In [24]:
n_iterations = 100

shap_vals_test = []
start_time = time.time()
for i in tqdm(range(n_iterations)):

    Inp_db = current_db_train.copy(deep=True)
    train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = i)
    
    X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]
    Y_val = Inp_db[Inp_db[Index_col].isin(val_idx)][target_col]

    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0)
    model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], verbose=False)
    explainer = shap.Explainer(model)
    shap_vals = explainer(X_val, Y_val).values
    shap_vals_mean = np.abs(shap_vals).mean(axis=0)
    shap_vals_test.append(shap_vals_mean)
    
durations['top_3_percent'] = time.time() - start_time
shap_val_summary = pd.DataFrame(data=np.array(shap_vals_test).mean(axis=0), columns=['shap'], index=current_db_train.columns[1:-1]).sort_values(by='shap',ascending=False)
selected_features = shap_val_summary.iloc[:int(len(current_db_train.columns[1:-1])*0.03),:].index.tolist()

with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'wt') as file:
    file.write(str(selected_features))
               
print(selected_features)
print(len(selected_features))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [02:19<00:00,  1.39s/it]

['pixel488', 'pixel514', 'pixel513', 'pixel516', 'pixel267', 'pixel544', 'pixel456', 'pixel323', 'pixel486', 'pixel239', 'pixel240', 'pixel211', 'pixel268', 'pixel455', 'pixel324', 'pixel546', 'pixel462', 'pixel485', 'pixel213', 'pixel458', 'pixel604', 'pixel543', 'pixel351']
23


## Powershap

In [25]:
start_time = time.time()
selector = PowerShap(
    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='powershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['powershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_PowerSHAP_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting powershap
Automatic mode enabled: Finding the minimal required powershap iterations for significance of 0.01.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: powershap requires 19 iterations; Adding 9  powershap iterations.


  0%|          | 0/9 [00:00<?, ?it/s]

Done!
selected features: ['pixel184', 'pixel185', 'pixel211', 'pixel212', 'pixel213', 'pixel215', 'pixel238', 'pixel239', 'pixel240', 'pixel243', 'pixel267', 'pixel268', 'pixel296', 'pixel297', 'pixel323', 'pixel324', 'pixel351', 'pixel352', 'pixel427', 'pixel436', 'pixel455', 'pixel456', 'pixel458', 'pixel460', 'pixel462', 'pixel463', 'pixel483', 'pixel486', 'pixel487', 'pixel488', 'pixel491', 'pixel513', 'pixel514', 'pixel515', 'pixel516', 'pixel518', 'pixel543', 'pixel544', 'pixel546', 'pixel548', 'pixel573', 'pixel604', 'pixel630']
43


## LLpowershap

In [52]:
start_time = time.time()
selector = PowerShap(model=XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='llpowershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['LLpowershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_LLpowershap_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting llpowershap
Automatic mode enabled: Finding the minimal required llpowershap iterations for significance of 0.01.


  0%|          | 0/20 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 44  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 64  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 80  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Done!
selected features: ['pixel152', 'pixel153', 'pixel154', 'pixel155', 'pixel156', 'pixel157', 'pixel159', 'pixel182', 'pixel183', 'pixel184', 'pixel185', 'pixel211', 'pixel212', 'pixel213', 'pixel238', 'pixel239', 'pixel240', 'pixel241', 'pixel242', 'pixel243', 'pixel267', 'pixel268', 'pixel269', 'pixel270', 'pixel294', 'pixel295', 'pixel296', 'pixel297', 'pixel302', 'pixel319', 'pixel322', 'pixel323', 'pixel324', 'pixel325', 'pixel347', 'pixel348', 'pixel350', 'pixel351', 'pixel352', 'pixel353', 'pixel358', 'pixel359', 'pixel375', 'pixel376', 'pixel377', 'pixel379', 'pixel380', 'pixel382', 'pixel402', 'pixel403', 'pixel404', 'pixel410', 'pixel414', 'pixel416', 'pixel427', 'pixel428', 'pixel429', 'pixel430', 'pixel431', 'pixel432', 'pixel433', 'pixel436', 'pixel437', 'pixel438', 'pixel454', 'pixel455', 'pixel456', 'pixel457', 'pixel458', 'pixel459', 'pixel460', 'pixel461', 'pixel462', 'pixel463', 'pixel464', 'pixel465', 'pixel483', 'pixel484', 'pixel485', 'pixel486', 'pixel487', 'p

## Borutashap

In [27]:
model = XGBClassifier(n_estimators=250, verbosity=0)#, #early_stopping_rounds=25)

# if classification is False it is a Regression problem
start_time = time.time()
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=current_db_train[list(current_db_train.columns.values[1:-1])], y=current_db_train[target_col], sample=False,
                        train_or_test = 'test', normalize=True,verbose=True)
dur = time.time() - start_time
durations['borutashap'] = dur

subset = Feature_Selector.Subset()
subset.to_csv(f'../results/{sub}/{ds}_borutashap_results.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

15 attributes confirmed important: ['pixel211', 'pixel514', 'pixel546', 'pixel323', 'pixel548', 'pixel579', 'pixel456', 'pixel604', 'pixel488', 'pixel267', 'pixel460', 'pixel240', 'pixel457', 'pixel351', 'pixel516']
740 attributes confirmed unimportant: ['pixel760', 'pixel2', 'pixel41', 'pixel765', 'pixel476', 'pixel169', 'pixel245', 'pixel15', 'pixel529', 'pixel466', 'pixel184', 'pixel20', 'pixel512', 'pixel336', 'pixel205', 'pixel384', 'pixel68', 'pixel237', 'pixel624', 'pixel360', 'pixel90', 'pixel179', 'pixel639', 'pixel300', 'pixel75', 'pixel539', 'pixel656', 'pixel673', 'pixel227', 'pixel654', 'pixel137', 'pixel423', 'pixel24', 'pixel397', 'pixel387', 'pixel695', 'pixel244', 'pixel338', 'pixel38', 'pixel82', 'pixel233', 'pixel261', 'pixel329', 'pixel524', 'pixel408', 'pixel288', 'pixel191', 'pixel200', 'pixel483', 'pixel526', 'pixel554', 'pixel510', 'pixel85', 'pixel608', 'pixel225', 'pixel473', 'pixel343', 'pixel94', 'pixel452', 'pixel198', 'pixel615', 'pixel50', 'pixel256', 'pi

# Shapicant

In [28]:
Inp_db = current_db_train.copy(deep=True)
train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = 0)

X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]

model = XGBClassifier(n_estimators=250,  verbosity=0) #,early_stopping_rounds=25)

start_time = time.time()
# This is the class (not its instance) of SHAP's TreeExplainer
explainer_type = shap.TreeExplainer

# Use PandasSelector with 100 iterations
selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

# Run the feature selection
# If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
# We can also provide additional parameters to the underlying estimator's fit method through estimator_params
selector.fit(X_train, Y_train, X_validation=X_val)#, estimator_params={"categorical_feature": None})

dur = time.time() - start_time
durations['shapicant'] = dur

# Just get the features list
selected_features = selector.get_features()

# We can also get the p-values as pandas Series
p_values = selector.p_values_

np.array(selected_features)
with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'wt') as file:
    file.write(str(selected_features))

Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [04:26<00:00,  2.66s/it]


In [30]:
with open(f'../results/{sub}/{ds}_shap_methods_durations.txt', 'wt') as file:
    file.write(str(durations))

# 10 fold cross validation

In [88]:
import warnings
warnings.filterwarnings('ignore')

method_names = {'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features', 'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }


for m in models:
    for folds in [10]:
    
        results_df : pd.DataFrame = None
        
        for method in method_names:
            if method == 'top_3_percent':
                with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                    s = file.read()
                selected_features = ast.literal_eval(s)
            elif method == 'llpowershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
            elif method == 'borutashap':
                df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
                selected_features = df.columns
            elif method == 'chi_squared':
                selected_features = list(current_db_train.columns.values[1:-1][np.where(chi2(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
            elif method == 'f_test':
                selected_features = list(current_db_train.columns.values[1:-1][np.where(f_classif(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
            elif method == 'powershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
            elif method == 'shapicant':
                with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                    s = file.read()
                selected_features = ast.literal_eval(s)
            elif method == 'default':
                selected_features = list(current_db.columns.values[1:-1])
        
            print(len(selected_features))
            # CB_model = CatBoostClassifier(verbose=False,iterations=250,random_seed=2,use_best_model=True)
            model = copy.deepcopy(models[m])
            scores_cv_train,scores_cv_test = benchmark_classification_cross_validation(Model = model,Inp_db = current_db_train.copy(deep=True),index_col=Index_col,folds=folds,RS=1,features = selected_features,target_col = target_col)
            
            print(method_names[method])
            print("TRAIN")
            for key in scores_cv_train:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_train[key]),4))+" ("+str(np.round(np.std(scores_cv_train[key]),4))+")")
            print(50*"-")
            print("TEST")
            for key in scores_cv_test:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),4))+" ("+str(np.round(np.std(scores_cv_test[key]),4))+")")
            print(50*"=")
        
            df_train = pd.DataFrame(scores_cv_train)
            df_train['data'] = 'train'
            df_train['method'] = method
        
            df_test = pd.DataFrame(scores_cv_test)
            df_test['data'] = 'test'
            df_test['method'] = method
        
            results_df = pd.concat([results_df, df_train, df_test])
        
        results_df.to_csv(f'../results/{sub}/{ds}_{folds}_fold_cross_val_results_with_{m}.csv', index=False)

        

123


10it [00:16,  1.67s/it]


LLpowershap
TRAIN
AUC: 1.0 (0.0)
MCC: 1.0 (0.0)
ACCURACY: 1.0 (0.0)
RECALL: 1.0 (0.0)
F1: 1.0 (0.0)
PRECISION: 1.0 (0.0)
AUC_PRC: 1.0 (0.0)
--------------------------------------------------
TEST
AUC: 0.9898 (0.005)
MCC: 0.9005 (0.0344)
ACCURACY: 0.9504 (0.0174)
RECALL: 0.9504 (0.0174)
F1: 0.9504 (0.0174)
PRECISION: 0.9507 (0.0171)
AUC_PRC: 0.9888 (0.0054)
43


10it [00:07,  1.26it/s]


powershap
TRAIN
AUC: 1.0 (0.0)
MCC: 0.9969 (0.0033)
ACCURACY: 0.9985 (0.0017)
RECALL: 0.9985 (0.0017)
F1: 0.9985 (0.0017)
PRECISION: 0.9985 (0.0017)
AUC_PRC: 1.0 (0.0)
--------------------------------------------------
TEST
AUC: 0.9807 (0.0073)
MCC: 0.8697 (0.0304)
ACCURACY: 0.935 (0.0155)
RECALL: 0.935 (0.0155)
F1: 0.935 (0.0155)
PRECISION: 0.9355 (0.0152)
AUC_PRC: 0.9808 (0.0067)
15


...

Chi²
TRAIN
AUC: 1.0 (0.0)
MCC: 0.9999 (0.0003)
ACCURACY: 1.0 (0.0001)
RECALL: 1.0 (0.0001)
F1: 1.0 (0.0001)
PRECISION: 1.0 (0.0001)
AUC_PRC: 1.0 (0.0)
--------------------------------------------------
TEST
AUC: 0.9875 (0.0061)
MCC: 0.8949 (0.0289)
ACCURACY: 0.9477 (0.0145)
RECALL: 0.9477 (0.0145)
F1: 0.9477 (0.0145)
PRECISION: 0.9479 (0.0144)
AUC_PRC: 0.9864 (0.0066)
405


10it [00:11,  1.14s/it]


F Test
TRAIN
AUC: 1.0 (0.0)
MCC: 0.9999 (0.0003)
ACCURACY: 1.0 (0.0001)
RECALL: 1.0 (0.0001)
F1: 1.0 (0.0001)
PRECISION: 1.0 (0.0001)
AUC_PRC: 1.0 (0.0)
--------------------------------------------------
TEST
AUC: 0.9871 (0.006)
MCC: 0.8989 (0.0188)
ACCURACY: 0.9496 (0.0095)
RECALL: 0.9496 (0.0095)
F1: 0.9496 (0.0095)
PRECISION: 0.95 (0.0094)
AUC_PRC: 0.9856 (0.0069)
23


10it [00:01,  7.23it/s]

top_3_percent
TRAIN
AUC: 0.9992 (0.0007)
MCC: 0.9825 (0.0099)
ACCURACY: 0.9912 (0.0049)
RECALL: 0.9912 (0.0049)
F1: 0.9912 (0.0049)
PRECISION: 0.9913 (0.0049)
AUC_PRC: 0.9993 (0.0007)
--------------------------------------------------
TEST
AUC: 0.969 (0.0105)
MCC: 0.8269 (0.0319)
ACCURACY: 0.9139 (0.016)
RECALL: 0.9139 (0.016)
F1: 0.9139 (0.016)
PRECISION: 0.9142 (0.016)
AUC_PRC: 0.9687 (0.009)


# Bootstrap testing

In [69]:

method_names = {'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features', 'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    
    results_df : pd.DataFrame = None
    
    for method in method_names:
        if method == 'top_3_percent':
            with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'llpowershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
        elif method == 'borutashap':
            df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
            selected_features = df.columns
        elif method == 'chi_squared':
            selected_features = list(current_db_train.columns.values[1:-1][np.where(chi2(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
        elif method == 'f_test':
            selected_features = list(current_db_train.columns.values[1:-1][np.where(f_classif(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
        elif method == 'powershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
        elif method == 'shapicant':
            with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'default':
            selected_features = list(current_db.columns.values[1:-1])
    
        X_train = current_db_train[selected_features]
        Y_train = current_db_train[target_col]
        
        X_test = current_db_test[selected_features]
        Y_test = current_db_test[target_col]
        
        model = copy.deepcopy(models[m])
        model.fit(X_train, Y_train)

        scores_cv_test = test_bootstrap_eval_class(Model = model, test_df = current_db_test.copy(deep=True),bootstrap_its=1000,RS=1,features = selected_features,target_col = target_col)

        print(method)
        print("TEST")
        for key in scores_cv_test:
            print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),3))+" ("+str(np.round(np.std(scores_cv_test[key]),3))+")")
    
        df_test = pd.DataFrame(scores_cv_test)
        df_test['data'] = 'test'
        df_test['method'] = method
    
        results_df = pd.concat([results_df, df_test])
    
    results_df.to_csv(f'../results/{sub}/{ds}_1000_bootstrap_results_with_{m}.csv', index=False)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 73.31it/s]


llpowershap
TEST
AUC: 0.988 (0.004)
MCC: 0.9 (0.018)
ACCURACY: 0.95 (0.009)
RECALL: 0.95 (0.009)
F1: 0.95 (0.009)
PRECISION: 0.95 (0.009)
AUC_PRC: 0.988 (0.004)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:12<00:00, 82.77it/s]


powershap
TEST
AUC: 0.976 (0.006)
MCC: 0.859 (0.021)
ACCURACY: 0.929 (0.011)
RECALL: 0.929 (0.011)
F1: 0.929 (0.011)
PRECISION: 0.93 (0.011)
AUC_PRC: 0.978 (0.006)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:11<00:00, 87.37it/s]


...

chi_squared
TEST
AUC: 0.989 (0.004)
MCC: 0.919 (0.016)
ACCURACY: 0.959 (0.008)
RECALL: 0.959 (0.008)
F1: 0.959 (0.008)
PRECISION: 0.96 (0.008)
AUC_PRC: 0.99 (0.004)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:19<00:00, 50.75it/s]


f_test
TEST
AUC: 0.989 (0.003)
MCC: 0.921 (0.016)
ACCURACY: 0.96 (0.008)
RECALL: 0.96 (0.008)
F1: 0.96 (0.008)
PRECISION: 0.961 (0.008)
AUC_PRC: 0.99 (0.003)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:17<00:00, 57.10it/s]

top_3_percent
TEST
AUC: 0.965 (0.007)
MCC: 0.831 (0.023)
ACCURACY: 0.916 (0.012)
RECALL: 0.916 (0.012)
F1: 0.916 (0.012)
PRECISION: 0.916 (0.012)
AUC_PRC: 0.967 (0.008)


# SCENE dataset

## Read dataset

In [75]:
current_db = pd.read_csv("../data/scene.csv", index_col=0)
current_db = current_db.reset_index()
current_db.loc[current_db.label==-1,"label"]=0
train_idx,val_idx = train_test_split(current_db["index"],test_size=0.25,random_state = 1)
current_db_train = current_db[current_db["index"].isin(train_idx)]
current_db_test = current_db[current_db["index"].isin(val_idx)]

target_col = "label"
Index_col = "index"

In [76]:
durations = {}
sub = 'xgb_250_git'
ds = 'scene'

## Top 3 percent

In [35]:
n_iterations = 100

shap_vals_test = []
start_time = time.time()
for i in tqdm(range(n_iterations)):

    Inp_db = current_db_train.copy(deep=True)
    train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = i)
    
    X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]
    Y_val = Inp_db[Inp_db[Index_col].isin(val_idx)][target_col]

    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0)
    model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], verbose=False)
    explainer = shap.Explainer(model)
    shap_vals = explainer(X_val, Y_val).values
    shap_vals_mean = np.abs(shap_vals).mean(axis=0)
    shap_vals_test.append(shap_vals_mean)
    
durations['top_3_percent'] = time.time() - start_time
shap_val_summary = pd.DataFrame(data=np.array(shap_vals_test).mean(axis=0), columns=['shap'], index=current_db_train.columns[1:-1]).sort_values(by='shap',ascending=False)
selected_features = shap_val_summary.iloc[:int(len(current_db_train.columns[1:-1])*0.03),:].index.tolist()

with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'wt') as file:
    file.write(str(selected_features))
               
print(selected_features)
print(len(selected_features))

100%|████████████████████████████████████████████████████████████████████████████████| 100/100 [00:49<00:00,  2.01it/s]

['attr88', 'attr244', 'attr241', 'attr226', 'attr229', 'attr80', 'attr219', 'attr81']
8


## Powershap

In [36]:
start_time = time.time()
selector = PowerShap(
    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='powershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['powershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_PowerSHAP_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting powershap
Automatic mode enabled: Finding the minimal required powershap iterations for significance of 0.01.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: powershap requires 23  iterations; The extra required iterations exceed the limit_automatic  threshold. powershap will add  10 powershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Done!
selected features: ['attr133', 'attr148', 'attr204', 'attr21', 'attr22', 'attr226', 'attr241', 'attr244', 'attr246', 'attr260', 'attr79', 'attr81', 'attr82', 'attr83', 'attr85', 'attr86', 'attr88', 'attr93', 'attr96', 'attr98']
20


## LLpowershap

In [57]:
start_time = time.time()
selector = PowerShap(model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='llpowershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['LLpowershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_LLpowershap_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting llpowershap
Automatic mode enabled: Finding the minimal required llpowershap iterations for significance of 0.01.


  0%|          | 0/20 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 44  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 78  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: llpowershap requires 74  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

Done!
selected features: ['attr102', 'attr103', 'attr104', 'attr133', 'attr148', 'attr155', 'attr169', 'attr200', 'attr207', 'attr208', 'attr209', 'attr21', 'attr219', 'attr22', 'attr223', 'attr226', 'attr229', 'attr237', 'attr238', 'attr239', 'attr240', 'attr241', 'attr244', 'attr245', 'attr246', 'attr253', 'attr26', 'attr270', 'attr271', 'attr39', 'attr44', 'attr45', 'attr47', 'attr48', 'attr57', 'attr59', 'attr66', 'attr67', 'attr68', 'attr72', 'attr74', 'attr75', 'attr76', 'attr78', 'attr79', 'attr80', 'attr81', 'attr82', 'attr83', 'attr85', 'attr86', 'attr88', 'attr89', 'attr90', 'attr91', 'attr92', 'attr93', 'attr96', 'attr97', 'attr98']
60


## Borutashap

In [38]:
model = XGBClassifier(n_estimators=250, verbosity=0)#, #early_stopping_rounds=25)

# if classification is False it is a Regression problem
start_time = time.time()
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=current_db_train[list(current_db_train.columns.values[1:-1])], y=current_db_train[target_col], sample=False,
                        train_or_test = 'test', normalize=True,verbose=True)
dur = time.time() - start_time
durations['borutashap'] = dur

subset = Feature_Selector.Subset()
subset.to_csv(f'../results/{sub}/{ds}_borutashap_results.csv', index=False)

  0%|          | 0/20 [00:00<?, ?it/s]

7 attributes confirmed important: ['attr244', 'attr26', 'attr253', 'attr75', 'attr88', 'attr241', 'attr79']
259 attributes confirmed unimportant: ['attr31', 'attr14', 'attr263', 'attr38', 'attr93', 'attr56', 'attr201', 'attr117', 'attr59', 'attr286', 'attr49', 'attr152', 'attr265', 'attr278', 'attr159', 'attr86', 'attr200', 'attr208', 'attr185', 'attr210', 'attr164', 'attr247', 'attr197', 'attr144', 'attr156', 'attr120', 'attr76', 'attr87', 'attr281', 'attr122', 'attr202', 'attr224', 'attr284', 'attr107', 'attr142', 'attr249', 'attr136', 'attr163', 'attr171', 'attr246', 'attr1', 'attr51', 'attr123', 'attr40', 'attr213', 'attr234', 'attr154', 'attr100', 'attr24', 'attr285', 'attr294', 'attr114', 'attr271', 'attr277', 'attr44', 'attr80', 'attr146', 'attr165', 'attr183', 'attr61', 'attr215', 'attr74', 'attr188', 'attr242', 'attr2', 'attr268', 'attr168', 'attr7', 'attr160', 'attr193', 'attr115', 'attr95', 'attr258', 'attr148', 'attr260', 'attr47', 'attr145', 'attr149', 'attr135', 'attr124'

# Shapicant

In [39]:
Inp_db = current_db_train.copy(deep=True)
train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = 0)

X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]

model = XGBClassifier(n_estimators=250,  verbosity=0) #,early_stopping_rounds=25)

start_time = time.time()
# This is the class (not its instance) of SHAP's TreeExplainer
explainer_type = shap.TreeExplainer

# Use PandasSelector with 100 iterations
selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

# Run the feature selection
# If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
# We can also provide additional parameters to the underlying estimator's fit method through estimator_params
selector.fit(X_train, Y_train, X_validation=X_val)#, estimator_params={"categorical_feature": None})

dur = time.time() - start_time
durations['shapicant'] = dur

# Just get the features list
selected_features = selector.get_features()

# We can also get the p-values as pandas Series
p_values = selector.p_values_

np.array(selected_features)
with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'wt') as file:
    file.write(str(selected_features))

Computing null SHAP values: 100%|████████████████████████████████████████████████████| 100/100 [02:13<00:00,  1.34s/it]


In [41]:
with open(f'../results/{sub}/{ds}_shap_methods_durations.txt', 'wt') as file:
    file.write(str(durations))

# 10 fold cross validation

In [80]:
import warnings
warnings.filterwarnings('ignore')

method_names = {'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features', 'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    for folds in [10]:
    
        results_df : pd.DataFrame = None
        
        for method in method_names:
            if method == 'top_3_percent':
                with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                    s = file.read()
                selected_features = ast.literal_eval(s)
            elif method == 'llpowershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
            elif method == 'borutashap':
                df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
                selected_features = df.columns
            elif method == 'chi_squared':
                selected_features = list(current_db_train.columns.values[1:-1][np.where(chi2(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
            elif method == 'f_test':
                selected_features = list(current_db_train.columns.values[1:-1][np.where(f_classif(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
            elif method == 'powershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
            elif method == 'shapicant':
                with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                    s = file.read()
                selected_features = ast.literal_eval(s)
            elif method == 'default':
                selected_features = list(current_db.columns.values[1:-1])
        
            # print(selected_features)
            # CB_model = CatBoostClassifier(verbose=False,iterations=250,random_seed=2,use_best_model=True)
            model = copy.deepcopy(models[m])
            scores_cv_train,scores_cv_test = benchmark_classification_cross_validation(Model = model,Inp_db = current_db_train.copy(deep=True),index_col=Index_col,folds=folds,RS=1,features = selected_features,target_col = target_col)
            
            print(method_names[method])
            print("TRAIN")
            for key in scores_cv_train:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_train[key]),4))+" ("+str(np.round(np.std(scores_cv_train[key]),4))+")")
            print(50*"-")
            print("TEST")
            for key in scores_cv_test:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),4))+" ("+str(np.round(np.std(scores_cv_test[key]),4))+")")
            print(50*"=")
        
            df_train = pd.DataFrame(scores_cv_train)
            df_train['data'] = 'train'
            df_train['method'] = method
        
            df_test = pd.DataFrame(scores_cv_test)
            df_test['data'] = 'test'
            df_test['method'] = method
        
            results_df = pd.concat([results_df, df_train, df_test])
        
        results_df.to_csv(f'../results/{sub}/{ds}_{folds}_fold_cross_val_results_with_{m}.csv', index=False)

        

10it [00:06,  1.43it/s]


LLpowershap
TRAIN
AUC: 0.9995 (0.0005)
MCC: 0.9704 (0.0236)
ACCURACY: 0.9914 (0.0068)
RECALL: 0.9914 (0.0068)
F1: 0.9913 (0.0069)
PRECISION: 0.9914 (0.0068)
AUC_PRC: 0.9979 (0.0023)
--------------------------------------------------
TEST
AUC: 0.9383 (0.0227)
MCC: 0.6515 (0.0635)
ACCURACY: 0.903 (0.0249)
RECALL: 0.903 (0.0249)
F1: 0.8962 (0.0296)
PRECISION: 0.9017 (0.023)
AUC_PRC: 0.7929 (0.0701)


10it [00:03,  2.99it/s]


powershap
TRAIN
AUC: 0.9896 (0.0065)
MCC: 0.8441 (0.07)
ACCURACY: 0.9559 (0.0193)
RECALL: 0.9559 (0.0193)
F1: 0.954 (0.0206)
PRECISION: 0.9557 (0.0196)
AUC_PRC: 0.9603 (0.0248)
--------------------------------------------------
TEST
AUC: 0.91 (0.0235)
MCC: 0.5488 (0.0726)
ACCURACY: 0.8786 (0.0298)
RECALL: 0.8786 (0.0298)
F1: 0.8682 (0.0353)
PRECISION: 0.8712 (0.0306)
AUC_PRC: 0.7242 (0.042)


10it [00:01,  5.07it/s]


...

[0]	validation_0-logloss:0.56907
[1]	validation_0-logloss:0.50894
[2]	validation_0-logloss:0.48184
[3]	validation_0-logloss:0.47000
[4]	validation_0-logloss:0.46791
[5]	validation_0-logloss:0.46600
[6]	validation_0-logloss:0.47134
[7]	validation_0-logloss:0.48231
[8]	validation_0-logloss:0.48391
[9]	validation_0-logloss:0.49165
[10]	validation_0-logloss:0.49144
[11]	validation_0-logloss:0.49516
[12]	validation_0-logloss:0.50026
[13]	validation_0-logloss:0.50643
[14]	validation_0-logloss:0.51260
[15]	validation_0-logloss:0.51472
[16]	validation_0-logloss:0.52115
[17]	validation_0-logloss:0.52282
[18]	validation_0-logloss:0.52747
[19]	validation_0-logloss:0.53261
[20]	validation_0-logloss:0.53447
[21]	validation_0-logloss:0.54054
[22]	validation_0-logloss:0.54091
[23]	validation_0-logloss:0.54256
[24]	validation_0-logloss:0.53853
[25]	validation_0-logloss:0.54174
[26]	validation_0-logloss:0.54423
[27]	validation_0-logloss:0.55768
[28]	validation_0-logloss:0.55979
[29]	validation_0-loglos

9it [00:00,  9.12it/s]

[0]	validation_0-logloss:0.56267
[1]	validation_0-logloss:0.49229
[2]	validation_0-logloss:0.45838
[3]	validation_0-logloss:0.43257
[4]	validation_0-logloss:0.41625
[5]	validation_0-logloss:0.41080
[6]	validation_0-logloss:0.40378
[7]	validation_0-logloss:0.39075
[8]	validation_0-logloss:0.39022
[9]	validation_0-logloss:0.38966
[10]	validation_0-logloss:0.39127
[11]	validation_0-logloss:0.39063
[12]	validation_0-logloss:0.38349
[13]	validation_0-logloss:0.38550
[14]	validation_0-logloss:0.38667
[15]	validation_0-logloss:0.38191
[16]	validation_0-logloss:0.37906
[17]	validation_0-logloss:0.37657
[18]	validation_0-logloss:0.37528
[19]	validation_0-logloss:0.37131
[20]	validation_0-logloss:0.37261
[21]	validation_0-logloss:0.37639
[22]	validation_0-logloss:0.37636
[23]	validation_0-logloss:0.36698
[24]	validation_0-logloss:0.36937
[25]	validation_0-logloss:0.37395
[26]	validation_0-logloss:0.37228
[27]	validation_0-logloss:0.37148
[28]	validation_0-logloss:0.37310
[29]	validation_0-loglos

10it [00:01,  9.10it/s]

top_3_percent
TRAIN
AUC: 0.9831 (0.0132)
MCC: 0.8207 (0.1033)
ACCURACY: 0.9478 (0.0295)
RECALL: 0.9478 (0.0295)
F1: 0.9474 (0.0298)
PRECISION: 0.9473 (0.03)
AUC_PRC: 0.9218 (0.0614)
--------------------------------------------------
TEST
AUC: 0.8591 (0.0399)
MCC: 0.4337 (0.0725)
ACCURACY: 0.8438 (0.0303)
RECALL: 0.8438 (0.0303)
F1: 0.8347 (0.0394)
PRECISION: 0.8353 (0.0358)
AUC_PRC: 0.5391 (0.0655)


# Bootstrap testing

In [81]:

method_names = {'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features', 'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    
    results_df : pd.DataFrame = None
    
    for method in method_names:
        if method == 'top_3_percent':
            with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'llpowershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
        elif method == 'borutashap':
            df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
            selected_features = df.columns
        elif method == 'chi_squared':
            selected_features = list(current_db_train.columns.values[1:-1][np.where(chi2(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
        elif method == 'f_test':
            selected_features = list(current_db_train.columns.values[1:-1][np.where(f_classif(current_db_train[current_db_train.columns.values[1:-1]],current_db_train[target_col])[1]<0.01)[0]])
        elif method == 'powershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
        elif method == 'shapicant':
            with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'default':
            selected_features = list(current_db.columns.values[1:-1])
    
    
        #print(selected_features)
    
        X_train = current_db_train[selected_features]
        Y_train = current_db_train[target_col]
        
        X_test = current_db_test[selected_features]
        Y_test = current_db_test[target_col]
        
        # CB_model = CatBoostClassifier(verbose=False,iterations=250,random_seed=2)#,per_float_feature_quantization=['1:border_count=1024']
        # CB_model.fit(X_train,Y_train)
        model = models[m]
        model.fit(X_train, Y_train)

        scores_cv_test = test_bootstrap_eval_class(Model = model, test_df = current_db_test.copy(deep=True),bootstrap_its=1000,RS=1,features = selected_features,target_col = target_col)
        
        #print(model)
        print(method)
        print("TEST")
        for key in scores_cv_test:
            print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),3))+" ("+str(np.round(np.std(scores_cv_test[key]),3))+")")
    
        df_test = pd.DataFrame(scores_cv_test)
        df_test['data'] = 'test'
        df_test['method'] = method
    
        results_df = pd.concat([results_df, df_test])
    
    results_df.to_csv(f'../results/{sub}/{ds}_1000_bootstrap_results_with_{m}.csv', index=False)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 96.86it/s]


llpowershap
TEST
AUC: 0.915 (0.016)
MCC: 0.557 (0.057)
ACCURACY: 0.886 (0.016)
RECALL: 0.886 (0.016)
F1: 0.875 (0.019)
PRECISION: 0.878 (0.018)
AUC_PRC: 0.7 (0.056)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 102.68it/s]


powershap
TEST
AUC: 0.885 (0.019)
MCC: 0.525 (0.058)
ACCURACY: 0.873 (0.017)
RECALL: 0.873 (0.017)
F1: 0.867 (0.018)
PRECISION: 0.865 (0.019)
AUC_PRC: 0.596 (0.064)


100%|█████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:09<00:00, 107.32it/s]


...

chi_squared
TEST
AUC: 0.909 (0.017)
MCC: 0.563 (0.054)
ACCURACY: 0.879 (0.016)
RECALL: 0.879 (0.016)
F1: 0.876 (0.017)
PRECISION: 0.875 (0.017)
AUC_PRC: 0.672 (0.058)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:14<00:00, 67.78it/s]


f_test
TEST
AUC: 0.918 (0.017)
MCC: 0.569 (0.056)
ACCURACY: 0.886 (0.016)
RECALL: 0.886 (0.016)
F1: 0.879 (0.018)
PRECISION: 0.879 (0.018)
AUC_PRC: 0.699 (0.058)


100%|██████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:13<00:00, 74.61it/s]


top_3_percent
TEST
AUC: 0.814 (0.025)
MCC: 0.34 (0.062)
ACCURACY: 0.823 (0.02)
RECALL: 0.823 (0.02)
F1: 0.816 (0.022)
PRECISION: 0.811 (0.023)
AUC_PRC: 0.458 (0.058)


# TDC Cancer Incidence dataset

## Read dataset

In [7]:
data_file_path = "../data/UKB_cancer.feather"

tdc_df = pd.read_feather(data_file_path)
tdc_df.index = tdc_df.userID
tdc_df = tdc_df.drop(columns='userID')

current_db = tdc_df

current_db = current_db.reset_index()
current_db['index'] = current_db.index

train_idx,val_idx = train_test_split(current_db["index"],test_size=0.25,random_state = 1)
current_db_train = current_db[current_db["index"].isin(train_idx)]
current_db_test = current_db[current_db["index"].isin(val_idx)]

target_col = "incident_cancer"
Index_col = "index"

In [8]:
durations = {}
sub = 'xgb_250'
ds = 'ukb_cancer'

## Top 3 percent

In [25]:
n_iterations = 100

shap_vals_test = []
start_time = time.time()
for i in tqdm(range(n_iterations)):

    Inp_db = current_db_train.copy(deep=True)
    train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = i)
    
    X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
    Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]
    Y_val = Inp_db[Inp_db[Index_col].isin(val_idx)][target_col]

    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0)
    model.fit(X_train, Y_train, eval_set=[(X_val, Y_val)], verbose=False)
    explainer = shap.Explainer(model)
    shap_vals = explainer(X_val, Y_val).values
    shap_vals_mean = np.abs(shap_vals).mean(axis=0)
    shap_vals_test.append(shap_vals_mean)
    
durations['top_3_percent'] = time.time() - start_time
shap_val_summary = pd.DataFrame(data=np.array(shap_vals_test).mean(axis=0), columns=['shap'], index=current_db_train.columns[1:-1]).sort_values(by='shap',ascending=False)
selected_features = shap_val_summary.iloc[:int(len(current_db_train.columns[1:-1])*0.03),:].index.tolist()

with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'wt') as file:
    file.write(str(selected_features))
               
print(selected_features)
print(len(selected_features))

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [5:58:00<00:00, 214.80s/it]

['x54', 'x20009', 'x20116', 'x50', 'x30500', 'x30850', 'x31', 'x30070', 'x30710', 'x30860', 'x30650', 'x30210', 'x48', 'x23102', 'x30270', 'x1697', 'x1259', 'x102', 'x30770', 'x20110__5', 'x30670', 'x30000', 'x30720', 'x129', 'x1835', 'x20107__13', 'x6142__2', 'x20015', 'x30730', 'x30600', 'x1747', 'x1807', 'x30890', 'x1647', 'x30090', 'x20107__101', 'x30110', 'x30750', 'x23110', 'x20160', 'x6141__2', 'x30080', 'x30510', 'x709', 'x1379', 'x4080', 'x30030', 'x137', 'x30690', 'x30700', 'x1797', 'x30130', 'x20110__101', 'x2217', 'x23107', 'x30620', 'x30290', 'x3063', 'x30040', 'x21002', 'x30820', 'x189', 'x30190', 'x20002__1065', 'x135', 'x30520', 'x30200', 'x30010', 'x20111__13', 'x20023', 'x30050', 'x30680', 'x136', 'x30790', 'x30810', 'x20002__1396', 'x20111__5', 'x30760', 'x20107__1', 'x21001', 'x6142__1', 'x30120', 'x400', 'x30220']
84


## Powershap

In [20]:
start_time = time.time()
selector = PowerShap(
    model = XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='powershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['powershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_PowerSHAP_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting powershap
Automatic mode enabled: Finding the minimal required powershap iterations for significance of 0.01.


  0%|          | 0/10 [00:00<?, ?it/s]

Automatic mode: powershap requires 23  iterations; The extra required iterations exceed the limit_automatic  threshold. powershap will add  10 powershap iterations and  re-evaluate.



Failed to converge on a solution.



  0%|          | 0/10 [00:00<?, ?it/s]


Failed to converge on a solution.



Done!
selected features: ['x1239', 'x1259', 'x129', 'x1647', 'x1807', 'x20009', 'x20107__13', 'x20116', 'x21002', 'x21022', 'x23102', 'x30000', 'x30070', 'x30090', 'x30120', 'x30210', 'x30500', 'x30600', 'x3063', 'x30650', 'x30690', 'x30710', 'x30720', 'x30730', 'x30770', 'x30850', 'x30860', 'x31', 'x50', 'x54']
30


## LLpowershap

In [21]:
start_time = time.time()
selector = PowerShap(model=XGBClassifier(n_estimators=250, early_stopping_rounds=25, verbosity=0),
    power_iterations=10,automatic=True, limit_automatic=10,verbose=True, method='llpowershap', target_col=target_col,index_col=Index_col
)
selector.fit(current_db_train.drop(columns=[Index_col,target_col]), current_db_train[target_col])
dur = time.time() - start_time
durations['LLpowershap'] = dur
t = selector._processed_shaps_df
t.reset_index().to_csv(f"../results/{sub}/{ds}_LLpowershap_results.csv",index=False)
print(f'selected features: {sorted(t[(t.p_value<0.01)].index.values)}')
print(len(t[(t.p_value<0.01)].index.values))

Starting llpowershap
Automatic mode enabled: Finding the minimal required llpowershap iterations for significance of 0.01.


  0%|          | 0/20 [00:00<?, ?it/s]

 99%|===================| 68494/68876 [00:58<00:00]        

Automatic mode: llpowershap requires 108  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

...

 99%|===================| 68018/68876 [01:10<00:00]        

Automatic mode: llpowershap requires 116  iterations; The extra required iterations exceed the limit_automatic  threshold. llpowershap will add  10 llpowershap iterations and  re-evaluate.


  0%|          | 0/10 [00:00<?, ?it/s]

 99%|===================| 68390/68876 [00:57<00:00]        

Done!
selected features: ['x1239', 'x1259', 'x129', 'x130', 'x1647', 'x1807', 'x1835', 'x20009', 'x20011', 'x20015', 'x20107__13', 'x20110__5', 'x20116', 'x21002', 'x21022', 'x23102', 'x23111', 'x30000', 'x30040', 'x30050', 'x30070', 'x30090', 'x30110', 'x30120', 'x30130', 'x30200', 'x30500', 'x30600', 'x3063', 'x30690', 'x30710', 'x30720', 'x30730', 'x30760', 'x30850', 'x30860', 'x31', 'x48', 'x50', 'x54', 'x709']
41


## Borutashap

In [58]:
model = XGBClassifier(n_estimators=250, verbosity=0)#, #early_stopping_rounds=25)

# if classification is False it is a Regression problem
start_time = time.time()
Feature_Selector = BorutaShap(model=model,
                              importance_measure='shap',
                              classification=True)

Feature_Selector.fit(X=current_db_train[list(current_db_train.columns.values[1:-2])], y=current_db_train[target_col], sample=False,
                        train_or_test = 'test', normalize=True,verbose=True)
dur = time.time() - start_time
durations['borutashap'] = dur

subset = Feature_Selector.Subset()
subset.to_csv(f'../results/{sub}/{ds}_borutashap_results.csv', index=False)

Warning there are missing values in your data !


  0%|          | 0/20 [00:00<?, ?it/s]

18 attributes confirmed important: ['x30730', 'x23102', 'x21022', 'x30070', 'x20116', 'x31', 'x1807', 'x30710', 'x20015', 'x1259', 'x30500', 'x30850', 'x30860', 'x50', 'x30210', 'x30770', 'x30650', 'x54']
2799 attributes confirmed unimportant: ['x20003__1140883518', 'x20003__1140860728', 'x20002__1067', 'x20004__1528', 'x20003__1140866026', 'x20003__1140874866', 'x20003__1140861434', 'x20003__1140876592', 'x20003__1140883494', 'x20003__1141165852', 'x6149__5', 'x20003__1140884314', 'x20003__1141164068', 'x20003__1141169516', 'x20004__1547', 'x20003__1140878378', 'x20003__1140870796', 'x20004__1572', 'x20004__1128', 'x20003__1140921938', 'x2030', 'x20003__1140878172', 'x20003__1140860328', 'x20002__1665', 'x20111__9', 'x20003__1140874160', 'x20002__1432', 'x20003__1140878734', 'x20003__1141200092', 'x20003__1140881856', 'x20003__1140858300', 'x20003__1140879658', 'x20003__1140879774', 'x20002__1266', 'x20003__1141152736', 'x20110__1', 'x20003__1140869034', 'x20003__1140870788', 'x20003_

## Shapicant

In [23]:
Inp_db = current_db_train.copy(deep=True)
train_idx,val_idx = train_test_split(Inp_db[Index_col],test_size=0.2,random_state = 0)

X_train = Inp_db[Inp_db[Index_col].isin(train_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
X_val = Inp_db[Inp_db[Index_col].isin(val_idx)].copy(deep=True).drop(columns=[Index_col,target_col])
Y_train = Inp_db[Inp_db[Index_col].isin(train_idx)][target_col]

model = XGBClassifier(n_estimators=250,  verbosity=0) #,early_stopping_rounds=25)

start_time = time.time()
# This is the class (not its instance) of SHAP's TreeExplainer
explainer_type = shap.TreeExplainer

# Use PandasSelector with 100 iterations
selector = shapicant.PandasSelector(model, explainer_type, random_state=42)

# Run the feature selection
# If we provide a validation set, SHAP values are computed on it, otherwise they are computed on the training set
# We can also provide additional parameters to the underlying estimator's fit method through estimator_params
selector.fit(X_train, Y_train, X_validation=X_val)#, estimator_params={"categorical_feature": None})

dur = time.time() - start_time
durations['shapicant'] = dur

# Just get the features list
selected_features = selector.get_features()

# We can also get the p-values as pandas Series
p_values = selector.p_values_

np.array(selected_features)
with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'wt') as file:
    file.write(str(selected_features))

Computing null SHAP values: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [29:32:53<00:00, 1063.73s/it]


In [60]:
with open(f'../results/{sub}/{ds}_shap_methods_durations.txt', 'wt') as file:
    file.write(str(durations))

# 10 fold cross validation

In [61]:
import warnings
warnings.filterwarnings('ignore')

method_names = {'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent', 'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    for folds in [10]:
    
        results_df : pd.DataFrame = None
        
        for method in method_names:
            if method == 'top_3_percent':
                with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                    s = file.read()
                selected_feautres = ast.literal_eval(s)
            elif method == 'llpowershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
            elif method == 'borutashap':
                df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
                selected_features = df.columns
            elif method == 'chi_squared':
                
                copy_df = current_db_train.fillna(0).copy(deep=True)
                copy_df.drop(columns=['index', 'userID'], inplace=True)
                copy_df = copy_df + copy_df.min(axis=0).abs()
                selected_features = list(copy_df.columns.values[:-1][np.where(chi2(copy_df.iloc[:, :-1],
                                                                                             copy_df[target_col])[1]<0.01)[0]])
            elif method == 'f_test':
                copy_df = current_db_train.fillna(0).copy(deep=True)
                copy_df.drop(columns=['index', 'userID'], inplace=True)
                #copy_df = copy_df + copy_df.min(axis=0).abs()
                selected_features = list(copy_df.columns.values[:-1][np.where(f_classif(copy_df.iloc[:, :-1],
                                                                                             copy_df[target_col])[1]<0.01)[0]])                
            elif method == 'powershap':
                processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
                selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
            elif method == 'shapicant':
                with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                    s = file.read()
                selected_features = ast.literal_eval(s)
            elif method == 'default':
                selected_features = list(current_db.columns.values[1:-2])
        
            # print(selected_features)
            # CB_model = CatBoostClassifier(verbose=False,iterations=250,random_seed=2,use_best_model=True)
            model = copy.deepcopy(models[m])
            scores_cv_train,scores_cv_test = benchmark_classification_cross_validation(Model = model,Inp_db = current_db_train.copy(deep=True),index_col=Index_col,folds=folds,RS=1,features = selected_features,target_col = target_col)
            
            print(method_names[method])
            print("TRAIN")
            for key in scores_cv_train:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_train[key]),4))+" ("+str(np.round(np.std(scores_cv_train[key]),4))+")")
            print(50*"-")
            print("TEST")
            for key in scores_cv_test:
                print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),4))+" ("+str(np.round(np.std(scores_cv_test[key]),4))+")")
            print(50*"=")
        
            df_train = pd.DataFrame(scores_cv_train)
            df_train['data'] = 'train'
            df_train['method'] = method
        
            df_test = pd.DataFrame(scores_cv_test)
            df_test['data'] = 'test'
            df_test['method'] = method
        
            results_df = pd.concat([results_df, df_train, df_test])
        
        results_df.to_csv(f'../results/{sub}/{ds}_{folds}_fold_cross_val_results_with_{m}.csv', index=False)



10it [39:48, 238.81s/it]


Chi²
TRAIN
AUC: 0.6947 (0.0054)
MCC: 0.071 (0.0081)
ACCURACY: 0.8945 (0.0002)
RECALL: 0.8945 (0.0002)
F1: 0.8453 (0.0004)
PRECISION: 0.8981 (0.0023)
AUC_PRC: 0.2489 (0.0127)
--------------------------------------------------
TEST
AUC: 0.6674 (0.003)
MCC: 0.0324 (0.0087)
ACCURACY: 0.8939 (0.0015)
RECALL: 0.8939 (0.0015)
F1: 0.8443 (0.0021)
PRECISION: 0.8611 (0.0153)
AUC_PRC: 0.1907 (0.0043)


10it [40:04, 240.48s/it]


F Test
TRAIN
AUC: 0.6969 (0.0069)
MCC: 0.0775 (0.0089)
ACCURACY: 0.8946 (0.0002)
RECALL: 0.8946 (0.0002)
F1: 0.8456 (0.0005)
PRECISION: 0.8985 (0.0016)
AUC_PRC: 0.2543 (0.0158)
--------------------------------------------------
TEST
AUC: 0.6684 (0.0031)
MCC: 0.0356 (0.0102)
ACCURACY: 0.894 (0.0015)
RECALL: 0.894 (0.0015)
F1: 0.8444 (0.0021)
PRECISION: 0.8629 (0.0122)
AUC_PRC: 0.1916 (0.0049)


10it [39:52, 239.21s/it]


...

borutashap
TRAIN
AUC: 0.7034 (0.0006)
MCC: 0.0264 (0.0033)
ACCURACY: 0.8939 (0.0002)
RECALL: 0.8939 (0.0002)
F1: 0.844 (0.0002)
PRECISION: 0.9041 (0.0023)
AUC_PRC: 0.2391 (0.002)
--------------------------------------------------
TEST
AUC: 0.6629 (0.0035)
MCC: 0.003 (0.006)
ACCURACY: 0.8938 (0.0015)
RECALL: 0.8938 (0.0015)
F1: 0.8438 (0.0021)
PRECISION: 0.8202 (0.0355)
AUC_PRC: 0.1821 (0.0044)


10it [01:14,  7.45s/it]


shapicant
TRAIN
AUC: 0.7094 (0.0005)
MCC: 0.026 (0.0026)
ACCURACY: 0.8939 (0.0002)
RECALL: 0.8939 (0.0002)
F1: 0.844 (0.0002)
PRECISION: 0.9032 (0.0036)
AUC_PRC: 0.2511 (0.001)
--------------------------------------------------
TEST
AUC: 0.6649 (0.0032)
MCC: 0.0031 (0.0078)
ACCURACY: 0.8938 (0.0015)
RECALL: 0.8938 (0.0015)
F1: 0.8438 (0.0021)
PRECISION: 0.8202 (0.0351)
AUC_PRC: 0.1826 (0.0036)


10it [22:34, 135.45s/it]

all features
TRAIN
AUC: 0.7275 (0.0009)
MCC: 0.0667 (0.0019)
ACCURACY: 0.8944 (0.0002)
RECALL: 0.8944 (0.0002)
F1: 0.8452 (0.0002)
PRECISION: 0.8955 (0.0007)
AUC_PRC: 0.2939 (0.0014)
--------------------------------------------------
TEST
AUC: 0.6662 (0.0035)
MCC: 0.0357 (0.0112)
ACCURACY: 0.894 (0.0014)
RECALL: 0.894 (0.0014)
F1: 0.8444 (0.0019)
PRECISION: 0.8649 (0.0139)
AUC_PRC: 0.1886 (0.0055)


# Bootstrap testing

In [63]:

method_names = {'chi_squared': 'Chi²', 'f_test': 'F Test', 'top_3_percent': 'top_3_percent', 'llpowershap': 'LLpowershap', 'powershap': 'powershap', 'borutashap': 'borutashap', 'shapicant': 'shapicant',
                'default': 'all features'
               }

models = {'cb_default': CatBoostClassifier(verbose=0,random_seed=2),
          'lgb_default': LGBMClassifier(verbose=0,random_seed=2)
         }

for m in models:
    
    results_df : pd.DataFrame = None
    
    for method in method_names:
        if method == 'top_3_percent':
            with open(f'../results/{sub}/{ds}_top_3_percent_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'llpowershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_LLpowershap_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values
        elif method == 'borutashap':
            df = pd.read_csv(f'../results/{sub}/{ds}_borutashap_results.csv')
            selected_features = df.columns
            
        elif method == 'chi_squared':
            
            copy_df = current_db_train.fillna(0).copy(deep=True)
            copy_df.drop(columns=['index', 'userID'], inplace=True)
            copy_df = copy_df + copy_df.min(axis=0).abs()
            selected_features = list(copy_df.columns.values[:-1][np.where(chi2(copy_df.iloc[:, :-1],
                                                                                         copy_df[target_col])[1]<0.01)[0]])
        elif method == 'f_test':
            copy_df = current_db_train.fillna(0).copy(deep=True)
            copy_df.drop(columns=['index', 'userID'], inplace=True)
            #copy_df = copy_df + copy_df.min(axis=0).abs()
            selected_features = list(copy_df.columns.values[:-1][np.where(f_classif(copy_df.iloc[:, :-1],
                                                                                         copy_df[target_col])[1]<0.01)[0]])  
        elif method == 'powershap':
            processed_shaps_df = pd.read_csv(f'../results/{sub}/{ds}_PowerSHAP_results.csv')
            selected_features = processed_shaps_df[(processed_shaps_df.p_value<0.01)]["index"].values     
        elif method == 'shapicant':
            with open(f'../results/{sub}/{ds}_shapicant_results.txt', 'rt') as file:
                s = file.read()
            selected_features = ast.literal_eval(s)
        elif method == 'default':
            selected_features = list(current_db.columns.values[1:-2])

    
        X_train = current_db_train[selected_features]
        Y_train = current_db_train[target_col]
        
        X_test = current_db_test[selected_features]
        Y_test = current_db_test[target_col]

        model = copy.deepcopy(models[m])
        model.fit(X_train, Y_train)

        scores_cv_test = test_bootstrap_eval_class(Model = model, test_df = current_db_test.copy(deep=True),bootstrap_its=1000,RS=1,features = selected_features,target_col = target_col)

        print(method)
        print("TEST")
        for key in scores_cv_test:
            print(str(key)+": "+str(np.round(np.mean(scores_cv_test[key]),3))+" ("+str(np.round(np.std(scores_cv_test[key]),3))+")")
    
        df_test = pd.DataFrame(scores_cv_test)
        df_test['data'] = 'test'
        df_test['method'] = method
    
        results_df = pd.concat([results_df, df_test])
    
    results_df.to_csv(f'../results/{sub}/{ds}_1000_bootstrap_results_with_{m}.csv', index=False)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [23:21<00:00,  1.40s/it]


chi_squared
TEST
AUC: 0.664 (0.003)
MCC: 0.037 (0.006)
ACCURACY: 0.894 (0.001)
RECALL: 0.894 (0.001)
F1: 0.845 (0.002)
PRECISION: 0.847 (0.006)
AUC_PRC: 0.184 (0.003)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [23:59<00:00,  1.44s/it]


f_test
TEST
AUC: 0.664 (0.003)
MCC: 0.035 (0.006)
ACCURACY: 0.894 (0.001)
RECALL: 0.894 (0.001)
F1: 0.845 (0.002)
PRECISION: 0.845 (0.006)
AUC_PRC: 0.183 (0.003)


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [17:11<00:00,  1.03s/it]


...

borutashap
TEST
AUC: 0.662 (0.003)
MCC: 0.007 (0.006)
ACCURACY: 0.894 (0.001)
RECALL: 0.894 (0.001)
F1: 0.845 (0.002)
PRECISION: 0.85 (0.038)
AUC_PRC: 0.18 (0.003)


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [19:21<00:00,  1.16s/it]


shapicant
TEST
AUC: 0.664 (0.003)
MCC: 0.003 (0.005)
ACCURACY: 0.894 (0.001)
RECALL: 0.894 (0.001)
F1: 0.845 (0.002)
PRECISION: 0.824 (0.02)
AUC_PRC: 0.181 (0.003)


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [3:18:54<00:00, 11.93s/it]

default
TEST
AUC: 0.665 (0.003)
MCC: 0.039 (0.006)
ACCURACY: 0.895 (0.001)
RECALL: 0.895 (0.001)
F1: 0.845 (0.002)
PRECISION: 0.865 (0.008)
AUC_PRC: 0.186 (0.003)
